Cat motivation before we begin.

![Cat Image](http://pngimg.com/uploads/cat/cat_PNG50538.png)

CatBoost is a high-performance open source library for gradient boosting on decision trees. More details can be found [here](https://catboost.ai/). This is one of my favourite gradient boosting library. 

In [ ]:
!pip install catboost

### Importing Libraries

In [ ]:
import numpy as np
import random
import os
import torch
from pathlib import Path
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from category_encoders import CountEncoder

### Seed

In [ ]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

### Reading Dataset

In [ ]:
path = Path('/kaggle/input/lish-moa')

In [ ]:
train = pd.read_csv(path / 'train_features.csv')
targets = pd.read_csv(path / 'train_targets_scored.csv')

test = pd.read_csv(path / 'test_features.csv')
sub = pd.read_csv(path / 'sample_submission.csv')

### Baseline Multiclassifier

In [ ]:
model = CatBoostClassifier(iterations=1,silent=False, random_state=0)
classifier = MultiOutputClassifier(model)
clf = Pipeline([('encode', CountEncoder(cols=["cp_type", "cp_dose"])), ('classify', classifier)])

In [ ]:
drop_id = ["sig_id"]

X_train = train.drop(drop_id, axis=1)
y_train = targets.drop(drop_id, axis=1)

In [ ]:
 clf.fit(X_train, y_train)

### Predictions

In [ ]:
X_test = test.drop(drop_id, axis=1)
preds = clf.predict_proba(X_test)
preds = np.array(preds)[:,:,1].T 

In [ ]:
control_mask = test['cp_type']=='ctl_vehicle'
X_test[control_mask] = 0

In [ ]:
sub.iloc[:,1:] = preds

In [ ]:
sub.to_csv('submission.csv', index=False)